In [1]:
import sys
import os

In [2]:
from importer import *
from utils import *

In [3]:
CLIPS_ROOT = raw_clip_data
SAVE_ROOT = os.path.join(save_clip_data, 'mask_pose')
SAVE_ROOT

'/media/haruyaishikawa/new_disk/save_clip_data/mask_pose'

### Check events

In [4]:
events = [event_names[i] for i in event_ids]
events

['meal time',
 'meeting',
 'coffee break',
 'nap',
 'cooking',
 'working',
 'tending to plants']

### Activate clip management

Testing for 'coffee break':

In [5]:
coffee = events[2]
clip_id = 0

#### Show images

In [6]:
dm = ShortClipManagement(coffee, CLIPS_ROOT, SAVE_ROOT)
clips = dm.change_event(coffee)

clip = clips[clip_id]
rgb_images = dm.get_sorted_rgb_images(clip)

Using Event coffee break
Available clips
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14']
Using Event coffee break
Available clips
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14']


In [7]:
%matplotlib widget

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [8]:
image_num = 100

image_path = os.path.join(dm.get_rgb_path(clip), rgb_images[image_num])

image = mpimg.imread(image_path)
plt.imshow(image)

FigureCanvasNbAgg()

In [9]:
manager = ClipsSavedDataManagement(event=coffee, root_path=SAVE_ROOT)

Using Event coffee break
Available clips
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14']


In [10]:
clips = manager.change_event(coffee)

Using Event coffee break
Available clips
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14']


In [11]:
clip = clips[clip_id]  # choose first clip

### Analysis

In [12]:
all_files = manager.get_sorted_npz_files(clip)
# files
print(len(all_files))

701


In [13]:
# starting point of clip
init = 100

length = len(all_files)
splits = 10
seg = int(length//splits)
print('segemnts: ', seg)

files = all_files[init:init+seg]

segemnts:  70


### Pose

In [14]:
from openpose import params, JointType

In [15]:
joint_val = JointType.Nose.value
print(JointType(joint_val))

JointType.Nose


In [16]:
joint = np.zeros((len(files), 3))
for i, f in enumerate(files):
    filename = os.path.join(manager.get_clip_directory(clip), f)
    poses, _ = poses_masks_from_npz(filename)

    if not(poses is None):
        pose_ids = list(poses.keys())
        points = poses[pose_ids[0]]

        point = points[joint_val]
        if (point == [0,0,0]).all():
            joint[i] = np.asarray([np.nan, np.nan, np.nan])
        else:
            joint[i] = point

roll_joint = np.rollaxis(joint, 1)

In [17]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from scipy.ndimage.filters import gaussian_filter

In [18]:
# in milimeters
y_min = -2000
y_max = 2000
x_min = -2000
x_max = 2000
bin_x = 100
bin_y = 100

sigma = 0.5

colormap = 'plasma'

def show_location_map(points, name):
    xedges = [i for i in range(x_min, x_max, bin_x)]
    yedges = [i for i in range(y_min, y_max, bin_y)]

    x = points[0][~np.isnan(points[0])]
    y = points[1][~np.isnan(points[1])]
    # z = points[2][~np.isnan(points[2])]

    H, xedges, yedges = np.histogram2d(x, y, bins=(xedges, yedges))
    H = H.T
    
    # normalize
    H = H/H.max()
    
    # gaussian filter?
    H = gaussian_filter(H, sigma=sigma)
    print(H.max())
    
    # color map?
    cmap = mpl.colors.ListedColormap(['grey','red'])

    fig = plt.figure()

    ax = fig.add_subplot(111, title=name,
                        aspect='equal')
    X, Y = np.meshgrid(xedges, yedges)
    ax.pcolormesh(X, Y, H, cmap=colormap, vmin=0.0, vmax=1.0)
    # ax.pcolor(X, Y, H)

    plt.show()


In [19]:
show_location_map(roll_joint, JointType(joint_val))

0.6639016538678132


FigureCanvasNbAgg()

#### Object

In [20]:
ob_name = 'chair'

calculate mean per each image

In [21]:
mean_of_points = []
for i, f in enumerate(files):
    filename = os.path.join(manager.get_clip_directory(clip), f)
    _, masks = poses_masks_from_npz(filename)

    if masks is None:
        continue

    item_number = [i for i, j in object_dict.items() if j == ob_name]

    for mask in masks:

        # TODO:
        # get string name
        # check if the first digit is some class
        # if so, append the mean
        id = int(mask.split('_')[0])

        if id in item_number:
            mean = masks[mask].mean(0)
            mean_of_points.append(mean)

mp = np.asarray(mean_of_points)
if mp.any() == False:
    print("no such object ", ob_name, " detected")

mp = np.rollaxis(mp, 1)

In [22]:
show_location_map(mp, ob_name)

0.6199070004603685


FigureCanvasNbAgg()

plot all points

In [23]:
all_points = None

for i, f in enumerate(files):
    filename = os.path.join(manager.get_clip_directory(clip), f)
    _, masks = poses_masks_from_npz(filename)
    
    if masks is None:
        continue

    item_index = coco_label_names.index(ob_name)
    
    for mask in masks:
        # get string name
        # check if the first digit is some class
        # if so, append the mean
        id = int(mask.split('_')[0])

        if id == item_index:
            if all_points is None:
                all_points = masks[mask]
            else:
                all_points = np.concatenate((all_points, masks[mask]), axis=0)

if all_points is None:
    print("no such object ", ob_name, " detected")


print(all_points.shape)
all_points = np.rollaxis(all_points, 1)

(129835, 3)


In [24]:
show_location_map(all_points, ob_name)

0.7947450043435796


FigureCanvasNbAgg()

### save as image?




In [25]:
from PIL import Image

In [26]:
def save_location_map(points, name):
    xedges = [i for i in range(x_min, x_max, bin_x)]
    yedges = [i for i in range(y_min, y_max, bin_y)]

    x = points[0][~np.isnan(points[0])]
    y = points[1][~np.isnan(points[1])]
    # z = points[2][~np.isnan(points[2])]

    H, xedges, yedges = np.histogram2d(x, y, bins=(xedges, yedges))
    H = H.T
    
    H = H/H.max()
    H = gaussian_filter(H, sigma=sigma)
    
    H_pix = (H * 255.9).astype(np.uint8)
    
    # y-axis becomes lowest to highest
    H_flipped = np.flipud(H_pix)
    
    img = Image.fromarray(H_flipped)
    img.save(f'{name}.png')

In [27]:
save_location_map(all_points, ob_name)

In [28]:
image = mpimg.imread(f'{ob_name}.png')
plt.imshow(image)

### Other plots

In [29]:
%matplotlib widget

In [30]:
def show_3d_grid(points, name):
    xedges = [i for i in range(x_min, x_max, bin_x)]
    yedges = [i for i in range(y_min, y_max, bin_y)]

    x = points[0][~np.isnan(points[0])]
    y = points[1][~np.isnan(points[1])]
    # z = points[2][~np.isnan(points[2])]

    H, xedges, yedges = np.histogram2d(x, y, bins=(xedges, yedges))
    H = H.T
    
    # normalize
    H = H/H.max()
    
    # gaussian filter?
    H = gaussian_filter(H, sigma=sigma)
    print(H.max())
    

    fig = plt.figure()
    
    ax = fig.gca(projection='3d', title=name)
    
    X, Y = np.meshgrid(xedges[:-1]+xedges[1:], yedges[:-1]+yedges[1:]) -(xedges[1]-xedges[0])
    
    xpos = X.flatten()*1./2
    ypos = Y.flatten()*1./2
    zpos = np.zeros_like(xpos)

    
    print(xedges[1])
    print(xedges[0])
    dx = xedges[1] - xedges[0]
    dy = yedges[1] - yedges[0]
    dz = H.flatten()

    cmap = mpl.cm.get_cmap('jet') # Get desired colormap - you can change this!
    max_height = np.max(dz)   # get range of colorbars so we can normalize
    min_height = np.min(dz)
    # scale each z to [0,1], and get their rgb values
    rgba = [cmap((k-min_height)/max_height) for k in dz] 
    
    print(xpos.shape)
    print(ypos.shape)
    print(zpos.shape)
    
    print(dx)
    print(dy)
    print(dz)
    
    ax.bar3d(xpos, ypos, zpos, dx, dy, dz, color=rgba, zsort='average')
    
    plt.show()

In [31]:
show_3d_grid(all_points, ob_name)

0.7947450043435796


FigureCanvasNbAgg()

-1900
-2000
(1521,)
(1521,)
(1521,)
100
100
[0. 0. 0. ... 0. 0. 0.]
